In [ ]:
#default_exp vision.rect_augment
#default_cls_lvl 3

# Rectangular computer vision augmentation

> Transforms to apply data augmentation to rectangular images

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.transform import *
from local.data.pipeline import *
from local.data.source import *
from local.data.core import *
from local.vision.core import *
from local.data.external import *
from local.notebook.showdoc import show_doc

In [ ]:
im = PILImage(PILImage.create(TEST_IMAGE).resize((30,20)))

In [ ]:
#export
@patch_property
def n_px(x: Image.Image): return x.size[0] * x.size[1]

In [ ]:
test_eq(im.n_px, 30*20)

In [ ]:
#export
@patch_property
def shape(x: Image.Image): return x.size[1],x.size[0]

In [ ]:
test_eq(im.shape, (20,30))

In [ ]:
#export
@patch
def reshape(x: Image.Image, h, w, resample=0): return x.resize((w,h), resample=resample)

In [ ]:
test_eq(im.reshape(12,10).shape, (12,10))

In [ ]:
#export
@patch_property
def aspect(x: Image.Image): return x.size[0]/x.size[1]

In [ ]:
test_eq(im.aspect, 30/20)

In [ ]:
#export
@patch
def resize_max(x: Image.Image, resample=0, max_px=None, max_h=None, max_w=None):
    h,w = x.shape
    if max_px and x.n_px>max_px: h,w = h*max_px/x.n_px,w*max_px/x.n_px
    if max_h and h>max_h: h,w = h*max_h/h,w*max_h/h
    if max_w and w>max_w: h,w = h*max_w/w,w*max_w/w
    return x.reshape(round(h), round(w), resample=resample)

In [ ]:
test_eq(im.resize_max(max_px=30*20).size, (30,20))
test_eq(im.resize_max(max_px=30*20/2).size, (15,10))
test_eq(im.resize_max(max_px=500, max_h=10, max_w=20).size, (15,10))
test_eq(im.resize_max(max_px=500, max_h=14, max_w=15).size, (15,10))
test_eq(im.resize_max(max_px=30*20/2, max_h=16, max_w=25).size, (15,10))

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_script.ipynb.
Converted 02_transforms.ipynb.
Converted 03_pipeline.ipynb.
Converted 04_data_external.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_source.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 09a_rect_augment.ipynb.
Converted 10_data_block.ipynb.
Converted 11_layers.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 30_text_core.ipynb.
Converted 90_notebook_core.ipynb.
Converted 91_notebook_export.ipynb.
Converted 92_notebook_showdoc.ipynb.
Converted 93_notebook_export2html.ipynb.
Converted 94_index.ipynb.
Converted 95_synth_learner.ipynb.
